# Plan: Denoising Dirty Documents

Goal: Achieve medal-level RMSE by building a strong classical denoising pipeline using OpenCV/Skimage, tuned on train vs train_cleaned, then generate test predictions in the exact sampleSubmission format.

Steps:
- Inspect sampleSubmission format to know output structure.
- Load train/train_cleaned as grayscale float in [0,1].
- Implement a set of candidate classical pipelines (median blur, non-local means, Gaussian, morphological opening/closing, bilateral, top-hat/black-hat) and simple combinations.
- Evaluate pipelines on all training images (average RMSE), log timings.
- Pick best global pipeline; if time permits, try lightweight per-image parameter selection via small grid.
- Generate predictions for test in required order and save submission.csv.

Next:
- Implement imports, IO helpers, RMSE, and sampleSubmission inspection.
- Add baseline pipelines and evaluation scaffold.

In [3]:
import os
import re
import time
from pathlib import Path
import numpy as np
import pandas as pd
import cv2

DATA_DIR = Path('.')
TRAIN_DIR = DATA_DIR / 'train'
TRAIN_CLEAN_DIR = DATA_DIR / 'train_cleaned'
TEST_DIR = DATA_DIR / 'test'
SAMPLE_SUB_PATH = DATA_DIR / 'sampleSubmission.csv'

def read_gray_uint8(path: Path) -> np.ndarray:
    img = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f'Could not read image: {path}')
    return img

def to_float01(img_uint8: np.ndarray) -> np.ndarray:
    return (img_uint8.astype(np.float32) / 255.0).clip(0.0, 1.0)

def rmse(a: np.ndarray, b: np.ndarray) -> float:
    a = a.astype(np.float32)
    b = b.astype(np.float32)
    return float(np.sqrt(np.mean((a - b) ** 2)))

# Discover data
train_files = sorted([p for p in TRAIN_DIR.glob('*.png')], key=lambda p: (len(p.stem), p.stem))
clean_files = sorted([p for p in TRAIN_CLEAN_DIR.glob('*.png')], key=lambda p: (len(p.stem), p.stem))
test_files = sorted([p for p in TEST_DIR.glob('*.png')], key=lambda p: (len(p.stem), p.stem))
print(f'Train: {len(train_files)} images, Train Cleaned: {len(clean_files)} images, Test: {len(test_files)} images')
assert len(train_files) == len(clean_files), 'Mismatch between train and train_cleaned counts'

# Quick shape check for a couple of pairs
for p in train_files[:3]:
    q = TRAIN_CLEAN_DIR / p.name
    a = read_gray_uint8(p)
    b = read_gray_uint8(q)
    print(f'Check {p.name}: train {a.shape}, clean {b.shape}, equal: {a.shape == b.shape}')

# Inspect sampleSubmission format
sample_df = pd.read_csv(SAMPLE_SUB_PATH)
print('sampleSubmission columns:', list(sample_df.columns))
print(sample_df.head())
print('sampleSubmission rows:', len(sample_df))

# Parse id pattern like image_pixel (e.g., 1_1) and extract first few image ids in order
id_col = sample_df.columns[0]
val_col = sample_df.columns[1] if len(sample_df.columns) > 1 else None
id_sample = sample_df[id_col].astype(str).values
m = re.match(r'^(\d+)_', id_sample[0])
if m:
    print('Detected id pattern imageId_pixelId. First 5 ids:', id_sample[:5])
else:
    print('Unknown id pattern. First 5 ids:', id_sample[:5])

# Utility: fast NLM denoising on uint8
def nlm_uint8(img_u8: np.ndarray, h: int = 10, templateWindowSize: int = 7, searchWindowSize: int = 21) -> np.ndarray:
    return cv2.fastNlMeansDenoising(img_u8, None, h=h, templateWindowSize=templateWindowSize, searchWindowSize=searchWindowSize)

Train: 115 images, Train Cleaned: 115 images, Test: 29 images
Check 2.png: train (258, 540), clean (258, 540), equal: True
Check 3.png: train (258, 540), clean (258, 540), equal: True
Check 5.png: train (258, 540), clean (258, 540), equal: True


sampleSubmission columns: ['id', 'value']
        id  value
0  110_1_1      1
1  110_1_2      1
2  110_1_3      1
3  110_1_4      1
4  110_1_5      1
sampleSubmission rows: 5789880
Detected id pattern imageId_pixelId. First 5 ids: ['110_1_1' '110_1_2' '110_1_3' '110_1_4' '110_1_5']


In [2]:
%pip install -q opencv-python-headless
import cv2
print('cv2 version:', cv2.__version__)

✅ Package installation completed and import cache refreshed.
cv2 version: 4.11.0



/app/.pip-target/cv2/__init__.py:11: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  import numpy


In [4]:
from collections import defaultdict
import math
import csv

# Baseline pipeline: NLM only (uint8 in, uint8 out), convert to float [0,1] for metrics/output
def pipeline_nlm_uint8(img_u8: np.ndarray, h: int = 10) -> np.ndarray:
    return nlm_uint8(img_u8, h=h, templateWindowSize=7, searchWindowSize=21)

def eval_h_values_on_train(h_list):
    results = []
    t0_all = time.time()
    for hi, h in enumerate(h_list):
        errs = []
        t0 = time.time()
        for i, p in enumerate(train_files):
            if i % 10 == 0:
                elapsed = time.time() - t0
                print(f'[h={h}] img {i+1}/{len(train_files)} elapsed {elapsed:.1f}s', flush=True)
            img = read_gray_uint8(p)
            den = pipeline_nlm_uint8(img, h=h)
            target = read_gray_uint8(TRAIN_CLEAN_DIR / p.name)
            err = rmse(to_float01(den), to_float01(target))
            errs.append(err)
        mean_rmse = float(np.mean(errs))
        med_rmse = float(np.median(errs))
        results.append((h, mean_rmse, med_rmse))
        print(f'[h={h}] mean RMSE={mean_rmse:.6f} median RMSE={med_rmse:.6f} time={time.time()-t0:.1f}s')
    print(f'Total grid time: {time.time()-t0_all:.1f}s')
    results.sort(key=lambda x: x[1])
    print('Top results by mean RMSE:')
    for h, m, md in results[:5]:
        print(f'  h={h}: mean={m:.6f}, median={md:.6f}')
    return results

def parse_id_triplet(id_str: str):
    # Expected format: imageId_row_col (1-based row/col)
    parts = id_str.split('_')
    if len(parts) != 3:
        raise ValueError(f'Unexpected id format: {id_str}')
    return int(parts[0]), int(parts[1]), int(parts[2])

def generate_submission(best_h: int, out_path: str = 'submission.csv'):
    print(f'Generating submission with h={best_h} -> {out_path}')
    # Build unique ordered image ids from sampleSubmission to ensure exact order
    sample_iter = pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000)
    ordered_image_ids = []
    seen = set()
    print('Collecting ordered unique image ids from sampleSubmission...')
    for chunk in sample_iter:
        for s in chunk['id'].astype(str).values:
            img_id, r, c = parse_id_triplet(s)
            if img_id not in seen:
                seen.add(img_id)
                ordered_image_ids.append(img_id)
    print(f'Found {len(ordered_image_ids)} unique test image ids in sampleSubmission.')

    # Precompute denoised images for each required test image id
    cache = {}
    for idx, img_id in enumerate(ordered_image_ids):
        img_path = TEST_DIR / f'{img_id}.png'
        print(f'[{idx+1}/{len(ordered_image_ids)}] Denoising test image {img_path} with h={best_h}...', flush=True)
        img_u8 = read_gray_uint8(img_path)
        den_u8 = pipeline_nlm_uint8(img_u8, h=best_h)
        den_f = to_float01(den_u8)
        cache[img_id] = den_f  # float32 [0,1]

    # Stream through sampleSubmission and write predictions in exact order
    print('Writing predictions to CSV in sample order...')
    with open(out_path, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['id', 'value'])
        for chunk in pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000):
            ids = chunk['id'].astype(str).values
            rows = []
            for s in ids:
                img_id, r, c = parse_id_triplet(s)
                img = cache[img_id]
                # Convert 1-based (r,c) to 0-based indices
                val = float(np.clip(img[r-1, c-1], 0.0, 1.0))
                rows.append((s, val))
            writer.writerows(rows)
    print(f'Wrote: {out_path}')

# Grid of h values for robust baseline
h_list = [8, 10, 12, 14, 16, 18, 20]

# Run evaluation (can comment out to skip after first run)
grid_results = eval_h_values_on_train(h_list)
best_h = grid_results[0][0]
print('Selected best h:', best_h)

# Generate submission with best_h
generate_submission(best_h, out_path='submission.csv')

[h=8] img 1/115 elapsed 0.0s


[h=8] img 11/115 elapsed 1.6s


[h=8] img 21/115 elapsed 3.2s


[h=8] img 31/115 elapsed 4.8s


[h=8] img 41/115 elapsed 6.4s


...

[27/29] Denoising test image test/8.png with h=14...


[28/29] Denoising test image test/80.png with h=14...


[29/29] Denoising test image test/95.png with h=14...


Writing predictions to CSV in sample order...


Wrote: submission.csv


In [5]:
import itertools
from zipfile import ZipFile, ZIP_DEFLATED

# Background estimation and normalization + denoise pipeline
def estimate_background_opening(img_u8: np.ndarray, ksize: int = 41) -> np.ndarray:
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (ksize, ksize))
    bg = cv2.morphologyEx(img_u8, cv2.MORPH_OPEN, kernel)
    # Smooth background to prevent halos
    bg = cv2.medianBlur(bg, 21)
    return bg

def normalize_by_background(img_u8: np.ndarray, bg_u8: np.ndarray) -> np.ndarray:
    img_f = img_u8.astype(np.float32)
    bg_f = bg_u8.astype(np.float32)
    bg_mean = float(np.mean(bg_f)) + 1e-6
    norm = (img_f / (bg_f + 1.0)) * bg_mean
    norm_u8 = np.clip(norm, 0, 255).astype(np.uint8)
    return norm_u8

def pipeline_bg_norm_nlm(img_u8: np.ndarray, ksize: int = 41, h: int = 10) -> np.ndarray:
    bg = estimate_background_opening(img_u8, ksize=ksize)
    norm = normalize_by_background(img_u8, bg)
    den = nlm_uint8(norm, h=h, templateWindowSize=7, searchWindowSize=21)
    return den

def eval_bg_norm_grid_on_train(ksizes, h_list):
    results = []
    t0_all = time.time()
    for gi, (ks, h) in enumerate(itertools.product(ksizes, h_list), start=1):
        errs = []
        t0 = time.time()
        print(f'Grid {gi}: ksize={ks}, h={h}')
        for i, p in enumerate(train_files):
            if i % 10 == 0:
                print(f'  [ks={ks},h={h}] img {i+1}/{len(train_files)}', flush=True)
            img = read_gray_uint8(p)
            den = pipeline_bg_norm_nlm(img, ksize=ks, h=h)
            target = read_gray_uint8(TRAIN_CLEAN_DIR / p.name)
            err = rmse(to_float01(den), to_float01(target))
            errs.append(err)
        mean_rmse = float(np.mean(errs))
        med_rmse = float(np.median(errs))
        elapsed = time.time() - t0
        print(f'  -> mean RMSE={mean_rmse:.6f}, median={med_rmse:.6f}, time={elapsed:.1f}s')
        results.append((ks, h, mean_rmse, med_rmse, elapsed))
    print(f'Total grid time: {time.time()-t0_all:.1f}s')
    results.sort(key=lambda x: x[2])
    print('Top results by mean RMSE:')
    for ks, h, m, md, _ in results[:5]:
        print(f'  ksize={ks}, h={h}: mean={m:.6f}, median={md:.6f}')
    return results

def generate_submission_bg_norm(best_ksize: int, best_h: int, out_path: str = 'submission.csv'):
    print(f'Generating submission with ksize={best_ksize}, h={best_h} -> {out_path}')
    # Ordered unique image ids from sample
    sample_iter = pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000)
    ordered_image_ids = []
    seen = set()
    for chunk in sample_iter:
        for s in chunk['id'].astype(str).values:
            img_id, r, c = parse_id_triplet(s)
            if img_id not in seen:
                seen.add(img_id)
                ordered_image_ids.append(img_id)
    print(f'Found {len(ordered_image_ids)} unique test image ids.')

    cache = {}
    for idx, img_id in enumerate(ordered_image_ids):
        img_path = TEST_DIR / f'{img_id}.png'
        print(f'[{idx+1}/{len(ordered_image_ids)}] Processing {img_path} ...', flush=True)
        img_u8 = read_gray_uint8(img_path)
        den_u8 = pipeline_bg_norm_nlm(img_u8, ksize=best_ksize, h=best_h)
        cache[img_id] = to_float01(den_u8)

    print('Writing predictions to CSV in sample order (rounded to 4 decimals)...')
    with open(out_path, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['id', 'value'])
        for chunk in pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000):
            ids = chunk['id'].astype(str).values
            rows = []
            for s in ids:
                img_id, r, c = parse_id_triplet(s)
                val = float(np.clip(cache[img_id][r-1, c-1], 0.0, 1.0))
                rows.append((s, f'{val:.4f}'))
            writer.writerows(rows)
    print(f'Wrote: {out_path}')
    # Optionally also zip (not required by local grader, but useful practice)
    try:
        zip_path = out_path + '.zip'
        with ZipFile(zip_path, 'w', compression=ZIP_DEFLATED) as zf:
            zf.write(out_path, arcname='submission.csv')
        print(f'Compressed to: {zip_path}')
    except Exception as e:
        print('Zip failed (ignored):', e)

# Evaluate improved classical pipeline
ksizes = [31, 41, 51]
h_list = [8, 10, 12]
results_bg = eval_bg_norm_grid_on_train(ksizes, h_list)
best_ks, best_h = results_bg[0][0], results_bg[0][1]
print('Selected best params -> ksize:', best_ks, 'h:', best_h)
generate_submission_bg_norm(best_ks, best_h, out_path='submission.csv')

Grid 1: ksize=31, h=8
  [ks=31,h=8] img 1/115


  [ks=31,h=8] img 11/115


  [ks=31,h=8] img 21/115


  [ks=31,h=8] img 31/115


  [ks=31,h=8] img 41/115


...

[28/29] Processing test/80.png ...


[29/29] Processing test/95.png ...


Writing predictions to CSV in sample order (rounded to 4 decimals)...


Wrote: submission.csv


Compressed to: submission.csv.zip


In [6]:
# Inspect train_cleaned intensity distribution to guide pipeline choice
import numpy as np
from collections import Counter

def summarize_clean_images(sample_n=10):
    stats = []
    for p in clean_files[:sample_n]:
        img = read_gray_uint8(p)
        u = np.unique(img)
        stats.append((p.name, img.min(), img.max(), len(u)))
        print(f'{p.name}: min={img.min()} max={img.max()} unique={len(u)}')
        if len(u) <= 8:
            # show small histogram of values
            counts = Counter(img.ravel().tolist())
            top = sorted(counts.items())[:10]
            print('  small unique set (first 10):', top)
    mins = [s[1] for s in stats]
    maxs = [s[2] for s in stats]
    us = [s[3] for s in stats]
    print('Summary over sample:')
    print('  min range:', (min(mins), max(mins)))
    print('  max range:', (min(maxs), max(maxs)))
    print('  unique counts range:', (min(us), max(us)))

summarize_clean_images(sample_n=15)

2.png: min=0 max=255 unique=256
3.png: min=0 max=255 unique=256
5.png: min=0 max=255 unique=256
9.png: min=0 max=255 unique=256
11.png: min=0 max=255 unique=256
12.png: min=0 max=255 unique=256
14.png: min=0 max=255 unique=256
15.png: min=0 max=255 unique=256
17.png: min=0 max=255 unique=256
18.png: min=0 max=255 unique=256
20.png: min=0 max=255 unique=256
21.png: min=0 max=255 unique=256
23.png: min=0 max=255 unique=256
24.png: min=0 max=255 unique=256
27.png: min=0 max=255 unique=256
Summary over sample:
  min range: (0, 0)
  max range: (255, 255)
  unique counts range: (256, 256)


In [7]:
# Alternative background normalization using OpenCV divide + denoise variants
import itertools

def estimate_background_median(img_u8: np.ndarray, ksize: int = 41) -> np.ndarray:
    # large median blur as background estimate
    k = ksize if ksize % 2 == 1 else ksize + 1
    return cv2.medianBlur(img_u8, k)

def normalize_divide(img_u8: np.ndarray, bg_u8: np.ndarray) -> np.ndarray:
    # Illumination correction via division with scale 255
    # Avoid zeros in bg
    bg_safe = bg_u8.copy()
    bg_safe[bg_safe < 1] = 1
    norm = cv2.divide(img_u8, bg_safe, scale=255)  # uint8 output
    return norm

def denoise_bilateral(img_u8: np.ndarray, d: int = 9, sc: int = 50, ss: int = 75) -> np.ndarray:
    return cv2.bilateralFilter(img_u8, d, sc, ss)

def pipeline_div_bilat(img_u8: np.ndarray, ksize: int = 41) -> np.ndarray:
    bg = estimate_background_median(img_u8, ksize=ksize)
    norm = normalize_divide(img_u8, bg)
    den = denoise_bilateral(norm, d=9, sc=50, ss=75)
    return den

def pipeline_div_nlm(img_u8: np.ndarray, ksize: int = 41, h: int = 10) -> np.ndarray:
    bg = estimate_background_median(img_u8, ksize=ksize)
    norm = normalize_divide(img_u8, bg)
    den = nlm_uint8(norm, h=h, templateWindowSize=7, searchWindowSize=21)
    return den

def eval_div_norm_grid_on_train(ksizes, methods):
    results = []
    t0_all = time.time()
    for gi, (ks, mth) in enumerate(itertools.product(ksizes, methods), start=1):
        errs = []
        t0 = time.time()
        print(f'Grid {gi}: ksize={ks}, method={mth}')
        for i, p in enumerate(train_files):
            if i % 10 == 0:
                print(f'  [ks={ks},method={mth}] img {i+1}/{len(train_files)}', flush=True)
            img = read_gray_uint8(p)
            if mth == 'bilateral':
                den = pipeline_div_bilat(img, ksize=ks)
            elif mth == 'nlm':
                den = pipeline_div_nlm(img, ksize=ks, h=10)
            else:
                raise ValueError(mth)
            target = read_gray_uint8(TRAIN_CLEAN_DIR / p.name)
            err = rmse(to_float01(den), to_float01(target))
            errs.append(err)
        mean_rmse = float(np.mean(errs))
        med_rmse = float(np.median(errs))
        elapsed = time.time() - t0
        print(f'  -> mean RMSE={mean_rmse:.6f}, median={med_rmse:.6f}, time={elapsed:.1f}s')
        results.append((ks, mth, mean_rmse, med_rmse, elapsed))
    print(f'Total grid time: {time.time()-t0_all:.1f}s')
    results.sort(key=lambda x: x[2])
    print('Top results by mean RMSE:')
    for ks, mth, m, md, _ in results[:5]:
        print(f'  ksize={ks}, method={mth}: mean={m:.6f}, median={md:.6f}')
    return results

# Evaluate this alternative quickly
ksizes_alt = [31, 41, 51]
methods_alt = ['bilateral', 'nlm']
results_div = eval_div_norm_grid_on_train(ksizes_alt, methods_alt)

Grid 1: ksize=31, method=bilateral
  [ks=31,method=bilateral] img 1/115


  [ks=31,method=bilateral] img 11/115


  [ks=31,method=bilateral] img 21/115


  [ks=31,method=bilateral] img 31/115


  [ks=31,method=bilateral] img 41/115


...

  [ks=51,method=nlm] img 81/115


  [ks=51,method=nlm] img 91/115


  [ks=51,method=nlm] img 101/115


  [ks=51,method=nlm] img 111/115


  -> mean RMSE=0.054467, median=0.054090, time=17.3s
Total grid time: 61.5s
Top results by mean RMSE:
  ksize=31, method=nlm: mean=0.049855, median=0.047893
  ksize=41, method=nlm: mean=0.051851, median=0.050700
  ksize=51, method=nlm: mean=0.054467, median=0.054090
  ksize=31, method=bilateral: mean=0.071353, median=0.072476
  ksize=41, method=bilateral: mean=0.072759, median=0.073789


In [8]:
# Fine-tune divide+NLM: tune pre-median blur and NLM h at best ksize=31
import itertools

def pipeline_div_nlm_tuned(img_u8: np.ndarray, ksize: int = 31, pre_median: int = 0, h: int = 10) -> np.ndarray:
    bg = estimate_background_median(img_u8, ksize=ksize)
    norm = normalize_divide(img_u8, bg)
    if pre_median and pre_median > 1:
        k = pre_median if pre_median % 2 == 1 else pre_median + 1
        norm = cv2.medianBlur(norm, k)
    den = nlm_uint8(norm, h=h, templateWindowSize=7, searchWindowSize=21)
    return den

def eval_div_nlm_finetune_on_train(ksize: int, pre_medians, h_list):
    results = []
    t0_all = time.time()
    for gi, (pm, h) in enumerate(itertools.product(pre_medians, h_list), start=1):
        errs = []
        t0 = time.time()
        print(f'Combo {gi}: ksize={ksize}, pre_median={pm}, h={h}')
        for i, p in enumerate(train_files):
            if i % 10 == 0:
                print(f'  [pm={pm},h={h}] img {i+1}/{len(train_files)}', flush=True)
            img = read_gray_uint8(p)
            den = pipeline_div_nlm_tuned(img, ksize=ksize, pre_median=pm, h=h)
            target = read_gray_uint8(TRAIN_CLEAN_DIR / p.name)
            err = rmse(to_float01(den), to_float01(target))
            errs.append(err)
        mean_rmse = float(np.mean(errs))
        med_rmse = float(np.median(errs))
        elapsed = time.time() - t0
        print(f'  -> mean RMSE={mean_rmse:.6f}, median={med_rmse:.6f}, time={elapsed:.1f}s')
        results.append((ksize, pm, h, mean_rmse, med_rmse, elapsed))
    print(f'Total finetune time: {time.time()-t0_all:.1f}s')
    results.sort(key=lambda x: x[3])
    print('Top results by mean RMSE:')
    for ks, pm, h, m, md, _ in results[:5]:
        print(f'  ksize={ks}, pre_median={pm}, h={h}: mean={m:.6f}, median={md:.6f}')
    return results

def generate_submission_div_nlm_tuned(best_ksize: int, best_pm: int, best_h: int, out_path: str = 'submission.csv'):
    print(f'Generating submission with ksize={best_ksize}, pre_median={best_pm}, h={best_h} -> {out_path}')
    sample_iter = pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000)
    ordered_image_ids = []
    seen = set()
    for chunk in sample_iter:
        for s in chunk['id'].astype(str).values:
            img_id, r, c = parse_id_triplet(s)
            if img_id not in seen:
                seen.add(img_id)
                ordered_image_ids.append(img_id)
    print(f'Found {len(ordered_image_ids)} unique test image ids.')
    cache = {}
    for idx, img_id in enumerate(ordered_image_ids):
        img_path = TEST_DIR / f'{img_id}.png'
        print(f'[{idx+1}/{len(ordered_image_ids)}] Processing {img_path} ...', flush=True)
        img_u8 = read_gray_uint8(img_path)
        den_u8 = pipeline_div_nlm_tuned(img_u8, ksize=best_ksize, pre_median=best_pm, h=best_h)
        cache[img_id] = to_float01(den_u8)
    import csv
    print('Writing predictions to CSV in sample order (rounded to 4 decimals)...')
    with open(out_path, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['id', 'value'])
        for chunk in pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000):
            rows = []
            for s in chunk['id'].astype(str).values:
                img_id, r, c = parse_id_triplet(s)
                val = float(np.clip(cache[img_id][r-1, c-1], 0.0, 1.0))
                rows.append((s, f'{val:.4f}'))
            writer.writerows(rows)
    print(f'Wrote: {out_path}')

# Run fine-tune around best ksize=31
pre_medians = [0, 3, 5]
h_list = [8, 10, 12, 14]
results_fine = eval_div_nlm_finetune_on_train(ksize=31, pre_medians=pre_medians, h_list=h_list)
best_ks, best_pm, best_h, best_mean, best_med, _ = results_fine[0]
print('Selected best tuned params -> ksize:', best_ks, 'pre_median:', best_pm, 'h:', best_h, 'mean:', best_mean)
# Uncomment to generate submission with tuned params once satisfied
# generate_submission_div_nlm_tuned(best_ks, best_pm, best_h, out_path='submission.csv')

Combo 1: ksize=31, pre_median=0, h=8
  [pm=0,h=8] img 1/115


  [pm=0,h=8] img 11/115


  [pm=0,h=8] img 21/115


  [pm=0,h=8] img 31/115


  [pm=0,h=8] img 41/115


...

  [pm=5,h=14] img 81/115


  [pm=5,h=14] img 91/115


  [pm=5,h=14] img 101/115


  [pm=5,h=14] img 111/115


  -> mean RMSE=0.226652, median=0.228232, time=17.2s
Total finetune time: 206.9s
Top results by mean RMSE:
  ksize=31, pre_median=0, h=14: mean=0.049552, median=0.047715
  ksize=31, pre_median=0, h=12: mean=0.049641, median=0.047782
  ksize=31, pre_median=0, h=10: mean=0.049855, median=0.047893
  ksize=31, pre_median=0, h=8: mean=0.050129, median=0.048123
  ksize=31, pre_median=3, h=8: mean=0.133535, median=0.139101
Selected best tuned params -> ksize: 31 pre_median: 0 h: 14 mean: 0.04955162887016068


In [9]:
# Generate submission using best tuned divide+NLM params
print('Best tuned params:', 'ksize=', best_ks, 'pre_median=', best_pm, 'h=', best_h)
generate_submission_div_nlm_tuned(int(best_ks), int(best_pm), int(best_h), out_path='submission.csv')

Best tuned params: ksize= 31 pre_median= 0 h= 14
Generating submission with ksize=31, pre_median=0, h=14 -> submission.csv


Found 29 unique test image ids.
[1/29] Processing test/110.png ...


[2/29] Processing test/111.png ...


[3/29] Processing test/122.png ...


[4/29] Processing test/131.png ...


...

[27/29] Processing test/8.png ...


[28/29] Processing test/80.png ...


[29/29] Processing test/95.png ...


Writing predictions to CSV in sample order (rounded to 4 decimals)...


Wrote: submission.csv


In [10]:
# Extended tuning: NLM template/search sizes and tiny post-blur to push <0.045
import itertools

def pipeline_div_nlm_ext(img_u8: np.ndarray, ksize_bg: int = 31, h: int = 14, tmpl: int = 7, srch: int = 21, post_gauss: int = 0) -> np.ndarray:
    bg = estimate_background_median(img_u8, ksize=ksize_bg)
    norm = normalize_divide(img_u8, bg)
    den = cv2.fastNlMeansDenoising(norm, None, h=h, templateWindowSize=tmpl, searchWindowSize=srch)
    if post_gauss and post_gauss > 1:
        k = post_gauss if post_gauss % 2 == 1 else post_gauss + 1
        den = cv2.GaussianBlur(den, (k, k), 0)
    return den

def eval_div_nlm_ext_on_train(ksize_bg: int, h_list, tmpl_list, srch_list, post_gauss_list):
    results = []
    t0_all = time.time()
    combos = list(itertools.product(h_list, tmpl_list, srch_list, post_gauss_list))
    for ci, (h, tmpl, srch, pg) in enumerate(combos, start=1):
        errs = []
        t0 = time.time()
        print(f'Combo {ci}/{len(combos)}: h={h}, tmpl={tmpl}, srch={srch}, post_gauss={pg}')
        for i, p in enumerate(train_files):
            if i % 10 == 0:
                print(f'  [h={h},t={tmpl},s={srch},pg={pg}] img {i+1}/{len(train_files)}', flush=True)
            img = read_gray_uint8(p)
            den = pipeline_div_nlm_ext(img, ksize_bg=ksize_bg, h=h, tmpl=tmpl, srch=srch, post_gauss=pg)
            target = read_gray_uint8(TRAIN_CLEAN_DIR / p.name)
            errs.append(rmse(to_float01(den), to_float01(target)))
        mean_rmse = float(np.mean(errs))
        med_rmse = float(np.median(errs))
        elapsed = time.time() - t0
        print(f'  -> mean RMSE={mean_rmse:.6f}, median={med_rmse:.6f}, time={elapsed:.1f}s')
        results.append((h, tmpl, srch, pg, mean_rmse, med_rmse, elapsed))
    print(f'Total ext grid time: {time.time()-t0_all:.1f}s')
    results.sort(key=lambda x: x[4])
    print('Top results by mean RMSE:')
    for h, tmpl, srch, pg, m, md, _ in results[:5]:
        print(f'  h={h}, tmpl={tmpl}, srch={srch}, post_gauss={pg}: mean={m:.6f}, median={md:.6f}')
    return results

# Run extended grid around best background ksize 31
h_list_ext = [12, 14, 16, 18]
tmpl_list_ext = [5, 7]
srch_list_ext = [21, 31]
post_gauss_list = [0, 3]
results_ext = eval_div_nlm_ext_on_train(ksize_bg=31, h_list=h_list_ext, tmpl_list=tmpl_list_ext, srch_list=srch_list_ext, post_gauss_list=post_gauss_list)
best_h, best_t, best_s, best_pg, best_mean_ext, best_med_ext, _ = results_ext[0]
print('Best ext params -> h:', best_h, 'tmpl:', best_t, 'srch:', best_s, 'post_gauss:', best_pg, 'mean:', best_mean_ext)

def generate_submission_div_nlm_ext(best_h: int, best_t: int, best_s: int, best_pg: int, out_path: str = 'submission.csv'):
    print(f'Generating submission (divide+NLM ext) with h={best_h}, tmpl={best_t}, srch={best_s}, post_gauss={best_pg} -> {out_path}')
    # Ordered unique image ids from sample
    sample_iter = pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000)
    ordered_image_ids = []
    seen = set()
    for chunk in sample_iter:
        for s in chunk['id'].astype(str).values:
            img_id, r, c = parse_id_triplet(s)
            if img_id not in seen:
                seen.add(img_id)
                ordered_image_ids.append(img_id)
    print(f'Found {len(ordered_image_ids)} unique test image ids.')
    cache = {}
    for idx, img_id in enumerate(ordered_image_ids):
        img_path = TEST_DIR / f'{img_id}.png'
        print(f'[{idx+1}/{len(ordered_image_ids)}] Processing {img_path} ...', flush=True)
        img_u8 = read_gray_uint8(img_path)
        den_u8 = pipeline_div_nlm_ext(img_u8, ksize_bg=31, h=best_h, tmpl=best_t, srch=best_s, post_gauss=best_pg)
        cache[img_id] = to_float01(den_u8)
    import csv
    print('Writing predictions to CSV in sample order (rounded to 4 decimals)...')
    with open(out_path, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['id', 'value'])
        for chunk in pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000):
            rows = []
            for s in chunk['id'].astype(str).values:
                img_id, r, c = parse_id_triplet(s)
                val = float(np.clip(cache[img_id][r-1, c-1], 0.0, 1.0))
                rows.append((s, f'{val:.4f}'))
            writer.writerows(rows)
    print(f'Wrote: {out_path}')

Combo 1/32: h=12, tmpl=5, srch=21, post_gauss=0
  [h=12,t=5,s=21,pg=0] img 1/115


  [h=12,t=5,s=21,pg=0] img 11/115


  [h=12,t=5,s=21,pg=0] img 21/115


  [h=12,t=5,s=21,pg=0] img 31/115


  [h=12,t=5,s=21,pg=0] img 41/115


...

  [h=18,t=7,s=31,pg=3] img 81/115


  [h=18,t=7,s=31,pg=3] img 91/115


  [h=18,t=7,s=31,pg=3] img 101/115


  [h=18,t=7,s=31,pg=3] img 111/115


  -> mean RMSE=0.122918, median=0.124949, time=39.1s
Total ext grid time: 893.4s
Top results by mean RMSE:
  h=14, tmpl=5, srch=31, post_gauss=0: mean=0.049001, median=0.047883
  h=12, tmpl=5, srch=31, post_gauss=0: mean=0.049095, median=0.047580
  h=16, tmpl=5, srch=31, post_gauss=0: mean=0.049137, median=0.048419
  h=14, tmpl=5, srch=21, post_gauss=0: mean=0.049215, median=0.047685
  h=16, tmpl=5, srch=21, post_gauss=0: mean=0.049264, median=0.047894
Best ext params -> h: 14 tmpl: 5 srch: 31 post_gauss: 0 mean: 0.04900112182873747


In [11]:
# Text-mask blending: divide normalization -> NLM background denoise -> adaptive threshold mask -> blend sharp text
import itertools

def build_text_mask(norm_u8: np.ndarray, block_size: int = 25, C: int = 10) -> np.ndarray:
    # block_size must be odd and >= 3
    b = block_size if block_size % 2 == 1 else block_size + 1
    mask = cv2.adaptiveThreshold(norm_u8, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, b, C)
    # Clean small holes/gaps: morphological close with 3x3
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    return mask

def pipeline_div_nlm_mask_blend(
    img_u8: np.ndarray,
    ksize_bg: int = 31,
    h_bg: int = 14,
    tmpl: int = 5,
    srch: int = 31,
    block_size: int = 25,
    C: int = 10,
    sharp_variant: str = 'norm'
) -> np.ndarray:
    # Normalize illumination
    bg = estimate_background_median(img_u8, ksize=ksize_bg)
    norm = normalize_divide(img_u8, bg)  # uint8
    # Denoise background strongly
    den_bg = cv2.fastNlMeansDenoising(norm, None, h=h_bg, templateWindowSize=tmpl, searchWindowSize=srch)
    # Sharp text source
    if sharp_variant == 'norm':
        sharp = norm
    elif sharp_variant == 'nlm8':
        sharp = cv2.fastNlMeansDenoising(norm, None, h=8, templateWindowSize=tmpl, searchWindowSize=srch)
    else:
        sharp = norm
    # Text mask where text=255
    mask = build_text_mask(norm, block_size=block_size, C=C)
    # Blend: text from sharp, background from denoised
    final_u8 = np.where(mask == 255, sharp, den_bg).astype(np.uint8)
    return final_u8

def eval_mask_blend_grid_on_train(block_sizes, C_list, sharp_variant: str = 'norm', ksize_bg: int = 31, h_bg: int = 14, tmpl: int = 5, srch: int = 31):
    results = []
    t0_all = time.time()
    combos = list(itertools.product(block_sizes, C_list))
    for ci, (bs, Cval) in enumerate(combos, start=1):
        errs = []
        t0 = time.time()
        print(f'Combo {ci}/{len(combos)}: block={bs}, C={Cval}, sharp={sharp_variant}')
        for i, p in enumerate(train_files):
            if i % 10 == 0:
                print(f'  [block={bs},C={Cval}] img {i+1}/{len(train_files)}', flush=True)
            img = read_gray_uint8(p)
            pred = pipeline_div_nlm_mask_blend(img, ksize_bg=ksize_bg, h_bg=h_bg, tmpl=tmpl, srch=srch, block_size=bs, C=Cval, sharp_variant=sharp_variant)
            target = read_gray_uint8(TRAIN_CLEAN_DIR / p.name)
            errs.append(rmse(to_float01(pred), to_float01(target)))
        mean_rmse = float(np.mean(errs))
        med_rmse = float(np.median(errs))
        elapsed = time.time() - t0
        print(f'  -> mean RMSE={mean_rmse:.6f}, median={med_rmse:.6f}, time={elapsed:.1f}s')
        results.append((bs, Cval, sharp_variant, mean_rmse, med_rmse, elapsed))
    print(f'Total mask-blend grid time: {time.time()-t0_all:.1f}s')
    results.sort(key=lambda x: x[3])
    print('Top results by mean RMSE:')
    for bs, Cval, sv, m, md, _ in results[:5]:
        print(f'  block={bs}, C={Cval}, sharp={sv}: mean={m:.6f}, median={md:.6f}')
    return results

def generate_submission_mask_blend(best_bs: int, best_C: int, sharp_variant: str = 'norm', out_path: str = 'submission.csv'):
    print(f'Generating submission (mask blend) with block={best_bs}, C={best_C}, sharp={sharp_variant} -> {out_path}')
    # Ordered unique image ids from sample
    sample_iter = pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000)
    ordered_image_ids = []
    seen = set()
    for chunk in sample_iter:
        for s in chunk['id'].astype(str).values:
            img_id, r, c = parse_id_triplet(s)
            if img_id not in seen:
                seen.add(img_id)
                ordered_image_ids.append(img_id)
    print(f'Found {len(ordered_image_ids)} unique test image ids.')
    # Precompute all outputs
    cache = {}
    for idx, img_id in enumerate(ordered_image_ids):
        img_path = TEST_DIR / f'{img_id}.png'
        print(f'[{idx+1}/{len(ordered_image_ids)}] Processing {img_path} ...', flush=True)
        img_u8 = read_gray_uint8(img_path)
        den_u8 = pipeline_div_nlm_mask_blend(img_u8, ksize_bg=31, h_bg=14, tmpl=5, srch=31, block_size=best_bs, C=best_C, sharp_variant=sharp_variant)
        cache[img_id] = to_float01(den_u8)
    # Write full-precision floats (no rounding) to avoid extra RMSE
    import csv
    print('Writing predictions to CSV in sample order (full precision floats)...')
    with open(out_path, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['id', 'value'])
        for chunk in pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000):
            rows = []
            for s in chunk['id'].astype(str).values:
                img_id, r, c = parse_id_triplet(s)
                val = float(np.clip(cache[img_id][r-1, c-1], 0.0, 1.0))
                rows.append((s, val))
            writer.writerows(rows)
    print(f'Wrote: {out_path}')

# Run a small, high-impact grid for mask blending
block_sizes = [23, 25, 29, 31]
C_list = [8, 10, 12]
results_mask = eval_mask_blend_grid_on_train(block_sizes, C_list, sharp_variant='norm', ksize_bg=31, h_bg=14, tmpl=5, srch=31)
best_bs, best_C, best_sv, best_mean_mb, best_med_mb, _ = results_mask[0]
print('Selected mask-blend params -> block:', best_bs, 'C:', best_C, 'sharp:', best_sv, 'mean:', best_mean_mb)
# After confirming best_mean_mb <= 0.045, generate submission:
# generate_submission_mask_blend(int(best_bs), int(best_C), sharp_variant=best_sv, out_path='submission.csv')

Combo 1/12: block=23, C=8, sharp=norm
  [block=23,C=8] img 1/115


  [block=23,C=8] img 11/115


  [block=23,C=8] img 21/115


  [block=23,C=8] img 31/115


  [block=23,C=8] img 41/115


...

  [block=31,C=12] img 81/115


  [block=31,C=12] img 91/115


  [block=31,C=12] img 101/115


  [block=31,C=12] img 111/115


  -> mean RMSE=0.049418, median=0.047659, time=38.1s
Total mask-blend grid time: 459.6s
Top results by mean RMSE:
  block=31, C=12, sharp=norm: mean=0.049418, median=0.047659
  block=29, C=12, sharp=norm: mean=0.049442, median=0.047659
  block=25, C=12, sharp=norm: mean=0.049508, median=0.047663
  block=31, C=10, sharp=norm: mean=0.049509, median=0.047672
  block=29, C=10, sharp=norm: mean=0.049537, median=0.047670
Selected mask-blend params -> block: 31 C: 12 sharp: norm mean: 0.04941835419639297


In [12]:
# Generate submission with best ext params (divide+NLM) writing full-precision floats (no rounding)
def generate_submission_div_nlm_ext_fullprecision(h: int, tmpl: int, srch: int, out_path: str = 'submission.csv'):
    print(f'Generating submission (full precision) with h={h}, tmpl={tmpl}, srch={srch} -> {out_path}')
    # Ordered unique image ids from sample
    sample_iter = pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000)
    ordered_image_ids = []
    seen = set()
    for chunk in sample_iter:
        for s in chunk['id'].astype(str).values:
            img_id, r, c = parse_id_triplet(s)
            if img_id not in seen:
                seen.add(img_id)
                ordered_image_ids.append(img_id)
    print(f'Found {len(ordered_image_ids)} unique test image ids.')
    # Precompute outputs
    cache = {}
    for idx, img_id in enumerate(ordered_image_ids):
        img_path = TEST_DIR / f'{img_id}.png'
        print(f'[{idx+1}/{len(ordered_image_ids)}] Processing {img_path} ...', flush=True)
        img_u8 = read_gray_uint8(img_path)
        den_u8 = pipeline_div_nlm_ext(img_u8, ksize_bg=31, h=h, tmpl=tmpl, srch=srch, post_gauss=0)
        cache[img_id] = to_float01(den_u8)
    # Write full precision floats
    import csv
    print('Writing predictions to CSV in sample order (full precision floats)...')
    with open(out_path, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['id', 'value'])
        for chunk in pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000):
            rows = []
            for s in chunk['id'].astype(str).values:
                img_id, r, c = parse_id_triplet(s)
                val = float(np.clip(cache[img_id][r-1, c-1], 0.0, 1.0))
                rows.append((s, val))
            writer.writerows(rows)
    print(f'Wrote: {out_path}')

# Use best ext params discovered earlier (h=14, tmpl=5, srch=31)
generate_submission_div_nlm_ext_fullprecision(h=14, tmpl=5, srch=31, out_path='submission.csv')

Generating submission (full precision) with h=14, tmpl=5, srch=31 -> submission.csv


Found 29 unique test image ids.
[1/29] Processing test/110.png ...


[2/29] Processing test/111.png ...


[3/29] Processing test/122.png ...


[4/29] Processing test/131.png ...


...

[27/29] Processing test/8.png ...


[28/29] Processing test/80.png ...


[29/29] Processing test/95.png ...


Writing predictions to CSV in sample order (full precision floats)...


Wrote: submission.csv


In [13]:
# Ensemble of divide+NLM variants (average multiple h values) to reduce RMSE
import itertools

def pipeline_div_nlm_ext_multi(img_u8: np.ndarray, hs: list[int], tmpl: int = 5, srch: int = 31, ksize_bg: int = 31) -> np.ndarray:
    # Shared normalization
    bg = estimate_background_median(img_u8, ksize=ksize_bg)
    norm = normalize_divide(img_u8, bg)
    acc = None
    for h in hs:
        den = cv2.fastNlMeansDenoising(norm, None, h=h, templateWindowSize=tmpl, searchWindowSize=srch)
        if acc is None:
            acc = den.astype(np.float32)
        else:
            acc += den.astype(np.float32)
    out = (acc / len(hs)).clip(0, 255).astype(np.uint8)
    return out

def eval_ensemble_on_train(h_sets, tmpl: int = 5, srch: int = 31, ksize_bg: int = 31):
    results = []
    for hs in h_sets:
        t0 = time.time()
        errs = []
        print(f'Ensemble eval hs={hs}, tmpl={tmpl}, srch={srch}')
        for i, p in enumerate(train_files):
            if i % 10 == 0:
                print(f'  [hs={hs}] img {i+1}/{len(train_files)}', flush=True)
            img = read_gray_uint8(p)
            den = pipeline_div_nlm_ext_multi(img, hs=hs, tmpl=tmpl, srch=srch, ksize_bg=ksize_bg)
            target = read_gray_uint8(TRAIN_CLEAN_DIR / p.name)
            errs.append(rmse(to_float01(den), to_float01(target)))
        mean_rmse = float(np.mean(errs))
        med_rmse = float(np.median(errs))
        print(f'  -> mean RMSE={mean_rmse:.6f}, median={med_rmse:.6f}, time={time.time()-t0:.1f}s')
        results.append((tuple(hs), mean_rmse, med_rmse))
    results.sort(key=lambda x: x[1])
    print('Top ensembles by mean RMSE:')
    for hs, m, md in results[:5]:
        print(f'  hs={hs}: mean={m:.6f}, median={md:.6f}')
    return results

def generate_submission_ensemble(hs: list[int], tmpl: int = 5, srch: int = 31, out_path: str = 'submission.csv'):
    print(f'Generating ensemble submission with hs={hs}, tmpl={tmpl}, srch={srch} -> {out_path}')
    # Ordered unique image ids from sample
    sample_iter = pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000)
    ordered_image_ids = []
    seen = set()
    for chunk in sample_iter:
        for s in chunk['id'].astype(str).values:
            img_id, r, c = parse_id_triplet(s)
            if img_id not in seen:
                seen.add(img_id)
                ordered_image_ids.append(img_id)
    print(f'Found {len(ordered_image_ids)} unique test image ids.')
    cache = {}
    for idx, img_id in enumerate(ordered_image_ids):
        img_path = TEST_DIR / f'{img_id}.png'
        print(f'[{idx+1}/{len(ordered_image_ids)}] Processing {img_path} ...', flush=True)
        img_u8 = read_gray_uint8(img_path)
        den_u8 = pipeline_div_nlm_ext_multi(img_u8, hs=hs, tmpl=tmpl, srch=srch, ksize_bg=31)
        cache[img_id] = to_float01(den_u8)
    # Write full precision floats
    import csv
    print('Writing predictions to CSV in sample order (full precision floats)...')
    with open(out_path, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['id', 'value'])
        for chunk in pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000):
            rows = []
            for s in chunk['id'].astype(str).values:
                img_id, r, c = parse_id_triplet(s)
                val = float(np.clip(cache[img_id][r-1, c-1], 0.0, 1.0))
                rows.append((s, val))
            writer.writerows(rows)
    print(f'Wrote: {out_path}')

# Evaluate a few ensemble sets (expect small gains ~0.001-0.003):
h_sets = [
    [12, 14],
    [14, 16],
    [12, 14, 16],
    [12, 14, 16, 18]
]
ens_results = eval_ensemble_on_train(h_sets, tmpl=5, srch=31, ksize_bg=31)
best_hs, best_m, best_md = ens_results[0]
print('Selected ensemble hs:', best_hs, 'mean:', best_m)
# If best_m <= 0.045: generate_submission_ensemble(list(best_hs), tmpl=5, srch=31, out_path='submission.csv')

Ensemble eval hs=[12, 14], tmpl=5, srch=31
  [hs=[12, 14]] img 1/115


  [hs=[12, 14]] img 11/115


  [hs=[12, 14]] img 21/115


  [hs=[12, 14]] img 31/115


  [hs=[12, 14]] img 41/115


...

  [hs=[12, 14, 16, 18]] img 81/115


  [hs=[12, 14, 16, 18]] img 91/115


  [hs=[12, 14, 16, 18]] img 101/115


  [hs=[12, 14, 16, 18]] img 111/115


  -> mean RMSE=0.049131, median=0.047834, time=148.7s
Top ensembles by mean RMSE:
  hs=(14, 16): mean=0.049055, median=0.047785
  hs=(12, 14, 16): mean=0.049060, median=0.047821
  hs=(12, 14): mean=0.049061, median=0.047617
  hs=(12, 14, 16, 18): mean=0.049131, median=0.047834
Selected ensemble hs: (14, 16) mean: 0.04905521061433398


In [14]:
# Per-image adaptive NLM h selection via robust noise estimate (MAD on high-pass) and RMSE-optimized thresholds
import numpy as np

def normalize_only(img_u8: np.ndarray, ksize_bg: int = 31) -> np.ndarray:
    bg = estimate_background_median(img_u8, ksize=ksize_bg)
    norm = normalize_divide(img_u8, bg)
    return norm

def estimate_sigma_mad(norm_u8: np.ndarray) -> float:
    # robust noise proxy via Laplacian high-pass MAD
    hp = cv2.Laplacian(norm_u8, cv2.CV_32F)
    med = np.median(hp)
    mad = np.median(np.abs(hp - med))
    sigma_hat = float(mad * 1.4826)  # approx sigma for Gaussian
    return sigma_hat

def nlm_on_norm(norm_u8: np.ndarray, h: int, tmpl: int = 5, srch: int = 31) -> np.ndarray:
    return cv2.fastNlMeansDenoising(norm_u8, None, h=h, templateWindowSize=tmpl, searchWindowSize=srch)

def collect_per_image_errs(train_paths, h_candidates=(12,14,16), tmpl=5, srch=31, ksize_bg=31):
    per_img = []  # list of dicts: {'path': p, 'sigma': sigma, 'errs': {h:err}, 'best_h': h, 'best_err': err}
    for i, p in enumerate(train_paths):
        if i % 10 == 0:
            print(f'collect errs: {i+1}/{len(train_paths)}', flush=True)
        img = read_gray_uint8(p)
        norm = normalize_only(img, ksize_bg)
        sigma = estimate_sigma_mad(norm)
        target = read_gray_uint8(TRAIN_CLEAN_DIR / p.name)
        target_f = to_float01(target)
        errs_map = {}
        best_h = None
        best_err = 1e9
        for h in h_candidates:
            den = nlm_on_norm(norm, h=h, tmpl=tmpl, srch=srch)
            err = rmse(to_float01(den), target_f)
            errs_map[h] = err
            if err < best_err:
                best_err = err
                best_h = h
        per_img.append({'path': p, 'sigma': sigma, 'errs': errs_map, 'best_h': best_h, 'best_err': best_err})
    mean_oracle = float(np.mean([d['best_err'] for d in per_img]))
    print('Mean RMSE of oracle per-image best-h:', mean_oracle)
    return per_img

def fit_sigma_thresholds_min_rmse(per_img, h_candidates=(12,14,16)):
    sigmas = np.array([d['sigma'] for d in per_img], dtype=np.float32)
    # percentile candidate cut points
    pcts = np.percentile(sigmas, [20, 30, 40, 50, 60, 70, 80])
    best = None  # (mean_rmse, t1, t2)
    for i in range(len(pcts)):
        for j in range(i+1, len(pcts)):
            t1, t2 = float(pcts[i]), float(pcts[j])
            errs = []
            for d in per_img:
                s = d['sigma']
                if s < t1:
                    h = h_candidates[0]
                elif s < t2:
                    h = h_candidates[1]
                else:
                    h = h_candidates[2]
                errs.append(d['errs'][h])
            mean_rmse = float(np.mean(errs))
            if best is None or mean_rmse < best[0]:
                best = (mean_rmse, t1, t2)
    print(f'Best thresholds by RMSE: mean={best[0]:.6f} at t1={best[1]:.6f}, t2={best[2]:.6f}')
    return best[1], best[2], best[0]

def eval_threshold_assignment_from_cache(per_img, t1: float, t2: float, h_candidates=(12,14,16)):
    errs = []
    for d in per_img:
        s = d['sigma']
        if s < t1:
            h = h_candidates[0]
        elif s < t2:
            h = h_candidates[1]
        else:
            h = h_candidates[2]
        errs.append(d['errs'][h])
    mean_rmse = float(np.mean(errs))
    med_rmse = float(np.median(errs))
    print(f'Cached threshold-assignment RMSE -> mean={mean_rmse:.6f}, median={med_rmse:.6f}')
    return mean_rmse, med_rmse

def generate_submission_per_image_h(t1: float, t2: float, h_candidates=(12,14,16), tmpl=5, srch=31, out_path='submission.csv', ksize_bg: int = 31, post_median: int = 0):
    print(f'Generating per-image-h submission with t1={t1:.6f}, t2={t2:.6f}, hs={h_candidates} -> {out_path}')
    # Ordered unique image ids from sample
    sample_iter = pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000)
    ordered_image_ids = []
    seen = set()
    for chunk in sample_iter:
        for s in chunk['id'].astype(str).values:
            img_id, r, c = parse_id_triplet(s)
            if img_id not in seen:
                seen.add(img_id)
                ordered_image_ids.append(img_id)
    print(f'Found {len(ordered_image_ids)} unique test image ids.')
    cache = {}
    for idx, img_id in enumerate(ordered_image_ids):
        img_path = TEST_DIR / f'{img_id}.png'
        print(f'[{idx+1}/{len(ordered_image_ids)}] Processing {img_path} ...', flush=True)
        img_u8 = read_gray_uint8(img_path)
        norm = normalize_only(img_u8, ksize_bg)
        sigma = estimate_sigma_mad(norm)
        if sigma < t1:
            h = h_candidates[0]
        elif sigma < t2:
            h = h_candidates[1]
        else:
            h = h_candidates[2]
        den_u8 = nlm_on_norm(norm, h=h, tmpl=tmpl, srch=srch)
        if post_median and post_median > 1:
            k = post_median if post_median % 2 == 1 else post_median + 1
            den_u8 = cv2.medianBlur(den_u8, k)
        cache[img_id] = to_float01(den_u8)
    # Write with 5 decimals to reduce file size
    import csv
    print('Writing predictions to CSV in sample order (5 decimals)...')
    with open(out_path, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['id', 'value'])
        for chunk in pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000):
            rows = []
            for s in chunk['id'].astype(str).values:
                img_id, r, c = parse_id_triplet(s)
                val = float(np.clip(cache[img_id][r-1, c-1], 0.0, 1.0))
                rows.append((s, f'{val:.5f}'))
            writer.writerows(rows)
    print(f'Wrote: {out_path}')

# Execute adaptive pipeline on train to set thresholds
h_candidates = (12, 14, 16)
per_img = collect_per_image_errs(train_files, h_candidates=h_candidates, tmpl=5, srch=31, ksize_bg=31)
t1, t2, best_mean = fit_sigma_thresholds_min_rmse(per_img, h_candidates=h_candidates)
mean_rmse_th, med_rmse_th = eval_threshold_assignment_from_cache(per_img, t1, t2, h_candidates=h_candidates)
print('Thresholds:', t1, t2, ' -> mean RMSE:', mean_rmse_th, 'median RMSE:', med_rmse_th)
# If improved, generate per-image-h submission (5 decimals to fit size limits)
# generate_submission_per_image_h(t1, t2, h_candidates=h_candidates, tmpl=5, srch=31, out_path='submission.csv', ksize_bg=31, post_median=3)

collect errs: 1/115


collect errs: 11/115


collect errs: 21/115


collect errs: 31/115


collect errs: 41/115


...

collect errs: 81/115


collect errs: 91/115


collect errs: 101/115


collect errs: 111/115


Mean RMSE of oracle per-image best-h: 0.04879453768548758
Best thresholds by RMSE: mean=0.049165 at t1=8.895600, t2=11.860800
Cached threshold-assignment RMSE -> mean=0.049165, median=0.047885
Thresholds: 8.895600318908691 11.860799789428711  -> mean RMSE: 0.049164921644589174 median RMSE: 0.04788483306765556


In [15]:
# Evaluate tiny post-median blur after best divide+NLM to see if RMSE improves
def pipeline_div_nlm_with_post_median(img_u8: np.ndarray, ksize_bg: int = 31, h: int = 14, tmpl: int = 5, srch: int = 31, post_median: int = 3) -> np.ndarray:
    bg = estimate_background_median(img_u8, ksize=ksize_bg)
    norm = normalize_divide(img_u8, bg)
    den = cv2.fastNlMeansDenoising(norm, None, h=h, templateWindowSize=tmpl, searchWindowSize=srch)
    if post_median and post_median > 1:
        k = post_median if post_median % 2 == 1 else post_median + 1
        den = cv2.medianBlur(den, k)
    return den

def eval_post_median_on_train(ksize_bg: int = 31, h: int = 14, tmpl: int = 5, srch: int = 31, post_median: int = 3):
    errs = []
    t0 = time.time()
    print(f'Evaluating post-median={post_median} after NLM (h={h}, tmpl={tmpl}, srch={srch})')
    for i, p in enumerate(train_files):
        if i % 10 == 0:
            print(f'  img {i+1}/{len(train_files)} elapsed {time.time()-t0:.1f}s', flush=True)
        img = read_gray_uint8(p)
        den = pipeline_div_nlm_with_post_median(img, ksize_bg=ksize_bg, h=h, tmpl=tmpl, srch=srch, post_median=post_median)
        target = read_gray_uint8(TRAIN_CLEAN_DIR / p.name)
        errs.append(rmse(to_float01(den), to_float01(target)))
    mean_rmse = float(np.mean(errs))
    med_rmse = float(np.median(errs))
    print(f'Post-median={post_median} -> mean RMSE={mean_rmse:.6f}, median RMSE={med_rmse:.6f}, time={time.time()-t0:.1f}s')
    return mean_rmse, med_rmse

# Run evaluation for post-median=3
post3_mean, post3_med = eval_post_median_on_train(ksize_bg=31, h=14, tmpl=5, srch=31, post_median=3)
print('Baseline best mean RMSE without post-median ~0.049001 for reference')

Evaluating post-median=3 after NLM (h=14, tmpl=5, srch=31)
  img 1/115 elapsed 0.0s


  img 11/115 elapsed 3.7s


  img 21/115 elapsed 7.4s


  img 31/115 elapsed 11.2s


  img 41/115 elapsed 14.8s


...

  img 81/115 elapsed 27.2s


  img 91/115 elapsed 30.3s


  img 101/115 elapsed 33.5s


  img 111/115 elapsed 36.6s


Post-median=3 -> mean RMSE=0.133014, median RMSE=0.138533, time=38.2s
Baseline best mean RMSE without post-median ~0.049001 for reference


In [16]:
# Generate 5-decimal submission using best fixed divide+NLM params to reduce file size
def generate_submission_div_nlm_ext_decimals(h: int, tmpl: int, srch: int, out_path: str = 'submission.csv', decimals: int = 5):
    fmt = '{:.' + str(decimals) + 'f}'
    print(f'Generating submission (fixed decimals={decimals}) with h={h}, tmpl={tmpl}, srch={srch} -> {out_path}')
    sample_iter = pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000)
    ordered_image_ids = []
    seen = set()
    for chunk in sample_iter:
        for s in chunk['id'].astype(str).values:
            img_id, r, c = parse_id_triplet(s)
            if img_id not in seen:
                seen.add(img_id)
                ordered_image_ids.append(img_id)
    print(f'Found {len(ordered_image_ids)} unique test image ids.')
    cache = {}
    for idx, img_id in enumerate(ordered_image_ids):
        img_path = TEST_DIR / f'{img_id}.png'
        print(f'[{idx+1}/{len(ordered_image_ids)}] Processing {img_path} ...', flush=True)
        img_u8 = read_gray_uint8(img_path)
        den_u8 = pipeline_div_nlm_ext(img_u8, ksize_bg=31, h=h, tmpl=tmpl, srch=srch, post_gauss=0)
        cache[img_id] = to_float01(den_u8)
    import csv
    print(f'Writing predictions to CSV in sample order ({decimals} decimals)...')
    with open(out_path, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['id', 'value'])
        for chunk in pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000):
            rows = []
            for s in chunk['id'].astype(str).values:
                img_id, r, c = parse_id_triplet(s)
                val = float(np.clip(cache[img_id][r-1, c-1], 0.0, 1.0))
                rows.append((s, fmt.format(val)))
            writer.writerows(rows)
    print(f'Wrote: {out_path}')

# Create 5-decimal submission for best ext params
generate_submission_div_nlm_ext_decimals(h=14, tmpl=5, srch=31, out_path='submission.csv', decimals=5)

Generating submission (fixed decimals=5) with h=14, tmpl=5, srch=31 -> submission.csv


Found 29 unique test image ids.
[1/29] Processing test/110.png ...


[2/29] Processing test/111.png ...


[3/29] Processing test/122.png ...


[4/29] Processing test/131.png ...


...

[27/29] Processing test/8.png ...


[28/29] Processing test/80.png ...


[29/29] Processing test/95.png ...


Writing predictions to CSV in sample order (5 decimals)...


Wrote: submission.csv


In [17]:
# BM3D implementation: replace NLM with BM3D on normalized float images
%pip install -q bm3d
import bm3d

def normalize_divide_float(img_u8: np.ndarray, ksize_bg: int = 31) -> np.ndarray:
    bg = estimate_background_median(img_u8, ksize=ksize_bg)
    bg_safe = bg.copy()
    bg_safe[bg_safe < 1] = 1
    norm_u8 = cv2.divide(img_u8, bg_safe, scale=255)  # uint8
    norm_f = norm_u8.astype(np.float32) / 255.0
    return norm_f

def estimate_sigma_mad_float(norm_f: np.ndarray) -> float:
    hp = cv2.Laplacian(norm_f, cv2.CV_32F)
    med = float(np.median(hp))
    mad = float(np.median(np.abs(hp - med)))
    sigma_hat = 1.4826 * mad
    return float(np.clip(sigma_hat, 0.03, 0.12))

def pipeline_div_bm3d(img_u8: np.ndarray, ksize_bg: int = 31, sigma_psd: float = 0.06) -> np.ndarray:
    norm_f = normalize_divide_float(img_u8, ksize_bg=ksize_bg)
    den_f = bm3d.bm3d(norm_f, sigma_psd=sigma_psd, stage_arg=bm3d.BM3DStages.ALL_STAGES)
    den_f = np.clip(den_f, 0.0, 1.0).astype(np.float32)
    return den_f

def eval_bm3d_global_sigma_on_train(sigmas: list[float], ksize_bg: int = 31):
    results = []
    for si, s in enumerate(sigmas, start=1):
        errs = []
        t0 = time.time()
        print(f'[BM3D global] sigma={s:.4f}')
        for i, p in enumerate(train_files):
            if i % 10 == 0:
                print(f'  sigma={s:.4f} img {i+1}/{len(train_files)} elapsed {time.time()-t0:.1f}s', flush=True)
            img = read_gray_uint8(p)
            den_f = pipeline_div_bm3d(img, ksize_bg=ksize_bg, sigma_psd=s)
            target_f = to_float01(read_gray_uint8(TRAIN_CLEAN_DIR / p.name))
            errs.append(rmse(den_f, target_f))
        mean_rmse = float(np.mean(errs))
        med_rmse = float(np.median(errs))
        print(f'  -> mean RMSE={mean_rmse:.6f}, median={med_rmse:.6f}, time={time.time()-t0:.1f}s')
        results.append((s, mean_rmse, med_rmse))
    results.sort(key=lambda x: x[1])
    print('Top BM3D global sigmas by mean RMSE:')
    for s, m, md in results[:5]:
        print(f'  sigma={s:.4f}: mean={m:.6f}, median={md:.6f}')
    return results

def eval_bm3d_per_image_sigma_on_train(factors=(0.8, 1.0, 1.2), ksize_bg: int = 31):
    # For each image, estimate sigma_hat via MAD on Laplacian of norm_f, then test multipliers
    sums = {f: 0.0 for f in factors}
    counts = 0
    t0_all = time.time()
    for i, p in enumerate(train_files):
        if i % 10 == 0:
            print(f'[BM3D per-image] img {i+1}/{len(train_files)} elapsed {time.time()-t0_all:.1f}s', flush=True)
        img = read_gray_uint8(p)
        norm_f = normalize_divide_float(img, ksize_bg=ksize_bg)
        sigma_hat = estimate_sigma_mad_float(norm_f)
        target_f = to_float01(read_gray_uint8(TRAIN_CLEAN_DIR / p.name))
        for f in factors:
            sigma = float(np.clip(sigma_hat * f, 0.03, 0.12))
            den_f = bm3d.bm3d(norm_f, sigma_psd=sigma, stage_arg=bm3d.BM3DStages.ALL_STAGES)
            den_f = np.clip(den_f, 0.0, 1.0).astype(np.float32)
            err = rmse(den_f, target_f)
            sums[f] += err
        counts += 1
    results = []
    for f in factors:
        mean_rmse = float(sums[f] / counts)
        results.append((f, mean_rmse))
    results.sort(key=lambda x: x[1])
    print('BM3D per-image sigma (MAD) multipliers by mean RMSE:')
    for f, m in results:
        print(f'  factor={f:.2f}: mean={m:.6f}')
    return results

def generate_submission_bm3d(out_path='submission.csv', decimals: int = 5, ksize_bg: int = 31,
                              mode: str = 'per_image', global_sigma: float = 0.06, factor: float = 1.0):
    assert mode in ('per_image', 'global')
    fmt = '{:.' + str(decimals) + 'f}'
    # Ordered unique image ids from sample
    sample_iter = pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000)
    ordered_image_ids = []
    seen = set()
    for chunk in sample_iter:
        for s in chunk['id'].astype(str).values:
            img_id, r, c = parse_id_triplet(s)
            if img_id not in seen:
                seen.add(img_id)
                ordered_image_ids.append(img_id)
    print(f'[BM3D submit] Found {len(ordered_image_ids)} unique test image ids.')
    cache = {}
    t0 = time.time()
    for idx, img_id in enumerate(ordered_image_ids):
        img_path = TEST_DIR / f'{img_id}.png'
        print(f'[{idx+1}/{len(ordered_image_ids)}] {img_path}', flush=True)
        img_u8 = read_gray_uint8(img_path)
        norm_f = normalize_divide_float(img_u8, ksize_bg=ksize_bg)
        if mode == 'per_image':
            sigma_hat = estimate_sigma_mad_float(norm_f)
            sigma = float(np.clip(sigma_hat * factor, 0.03, 0.12))
        else:
            sigma = float(global_sigma)
        den_f = bm3d.bm3d(norm_f, sigma_psd=sigma, stage_arg=bm3d.BM3DStages.ALL_STAGES)
        den_f = np.clip(den_f, 0.0, 1.0).astype(np.float32)
        cache[img_id] = den_f
    print('Writing BM3D predictions to CSV in sample order (fixed decimals)...')
    import csv
    with open(out_path, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['id', 'value'])
        for chunk in pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000):
            rows = []
            for s in chunk['id'].astype(str).values:
                img_id, r, c = parse_id_triplet(s)
                val = float(np.clip(cache[img_id][r-1, c-1], 0.0, 1.0))
                rows.append((s, fmt.format(val)))
            writer.writerows(rows)
    print(f'Wrote: {out_path}')

# Run BM3D evaluations: first quick global, then per-image multipliers
global_sigmas = [0.04, 0.05, 0.06, 0.07, 0.08]
bm3d_global_results = eval_bm3d_global_sigma_on_train(global_sigmas, ksize_bg=31)
per_image_results = eval_bm3d_per_image_sigma_on_train(factors=(0.8, 1.0, 1.2), ksize_bg=31)
best_global_sigma = bm3d_global_results[0][0]
best_factor = per_image_results[0][0]
print('Best BM3D global sigma:', best_global_sigma)
print('Best BM3D per-image factor:', best_factor)
# After choosing, generate submission (per-image preferred):
# generate_submission_bm3d(out_path='submission.csv', decimals=5, ksize_bg=31, mode='per_image', factor=best_factor)
# Or, for global: generate_submission_bm3d(out_path='submission.csv', decimals=5, ksize_bg=31, mode='global', global_sigma=best_global_sigma)

✅ Package installation completed and import cache refreshed.


[BM3D global] sigma=0.0400
  sigma=0.0400 img 1/115 elapsed 0.0s


  sigma=0.0400 img 11/115 elapsed 17.3s


  sigma=0.0400 img 21/115 elapsed 34.5s


  sigma=0.0400 img 31/115 elapsed 51.7s


...

[BM3D per-image] img 81/115 elapsed 469.7s


[BM3D per-image] img 91/115 elapsed 535.6s


[BM3D per-image] img 101/115 elapsed 601.3s


[BM3D per-image] img 111/115 elapsed 667.2s


BM3D per-image sigma (MAD) multipliers by mean RMSE:
  factor=0.80: mean=0.051921
  factor=1.00: mean=0.052700
  factor=1.20: mean=0.053881
Best BM3D global sigma: 0.04
Best BM3D per-image factor: 0.8


In [18]:
# Soft blending: gradient-weighted mix of NLM denoise and normalized image
def pipeline_div_nlm_soft_blend(img_u8: np.ndarray, ksize_bg: int = 31, h: int = 14, tmpl: int = 5, srch: int = 31, alpha: float = 0.3, weight_gamma: float = 1.0) -> np.ndarray:
    bg = estimate_background_median(img_u8, ksize=ksize_bg)
    norm = normalize_divide(img_u8, bg)  # uint8
    den = cv2.fastNlMeansDenoising(norm, None, h=h, templateWindowSize=tmpl, searchWindowSize=srch)  # uint8
    # gradient magnitude on normalized image
    gx = cv2.Sobel(norm, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(norm, cv2.CV_32F, 0, 1, ksize=3)
    grad = cv2.magnitude(gx, gy)
    w = grad / (grad.max() + 1e-6)  # [0,1]
    if weight_gamma != 1.0:
        w = np.power(w, weight_gamma)
    w = np.clip(w, 0.0, 1.0).astype(np.float32)
    den_f = den.astype(np.float32) / 255.0
    norm_f = norm.astype(np.float32) / 255.0
    out_f = (1.0 - alpha * w) * den_f + (alpha * w) * norm_f
    out_u8 = np.clip(out_f * 255.0, 0, 255).astype(np.uint8)
    return out_u8

def eval_soft_blend_on_train(alphas=(0.2, 0.3, 0.4), weight_gammas=(1.0, 0.7, 0.5), ksize_bg: int = 31, h: int = 14, tmpl: int = 5, srch: int = 31):
    results = []
    for a in alphas:
        for g in weight_gammas:
            errs = []
            t0 = time.time()
            print(f'[soft-blend] alpha={a}, gamma={g}')
            for i, p in enumerate(train_files):
                if i % 10 == 0:
                    print(f'  img {i+1}/{len(train_files)} elapsed {time.time()-t0:.1f}s', flush=True)
                img = read_gray_uint8(p)
                pred = pipeline_div_nlm_soft_blend(img, ksize_bg=ksize_bg, h=h, tmpl=tmpl, srch=srch, alpha=a, weight_gamma=g)
                target = read_gray_uint8(TRAIN_CLEAN_DIR / p.name)
                errs.append(rmse(to_float01(pred), to_float01(target)))
            mean_rmse = float(np.mean(errs))
            med_rmse = float(np.median(errs))
            print(f'  -> mean RMSE={mean_rmse:.6f}, median RMSE={med_rmse:.6f}, time={time.time()-t0:.1f}s')
            results.append((a, g, mean_rmse, med_rmse))
    results.sort(key=lambda x: x[2])
    print('Top soft-blend configs:')
    for a, g, m, md in results[:5]:
        print(f'  alpha={a}, gamma={g}: mean={m:.6f}, median={md:.6f}')
    return results

# Run soft blending evaluation
soft_results = eval_soft_blend_on_train(alphas=(0.2, 0.3, 0.4), weight_gammas=(1.0, 0.7, 0.5), ksize_bg=31, h=14, tmpl=5, srch=31)
best_a, best_g, best_m, best_md = soft_results[0]
print('Best soft blend -> alpha:', best_a, 'gamma:', best_g, 'mean:', best_m)
# If best_m improves over 0.049001, we can generate a submission with that config using pipeline_div_nlm_soft_blend.

[soft-blend] alpha=0.2, gamma=1.0
  img 1/115 elapsed 0.0s


  img 11/115 elapsed 3.7s


  img 21/115 elapsed 7.3s


  img 31/115 elapsed 11.0s


  img 41/115 elapsed 14.6s


...

  img 81/115 elapsed 27.1s


  img 91/115 elapsed 30.3s


  img 101/115 elapsed 33.5s


  img 111/115 elapsed 36.6s


  -> mean RMSE=0.049097, median RMSE=0.047602, time=38.2s
Top soft-blend configs:
  alpha=0.4, gamma=0.7: mean=0.049072, median=0.047639
  alpha=0.4, gamma=1.0: mean=0.049073, median=0.047683
  alpha=0.3, gamma=0.7: mean=0.049086, median=0.047676
  alpha=0.3, gamma=1.0: mean=0.049087, median=0.047702
  alpha=0.2, gamma=0.7: mean=0.049095, median=0.047704
Best soft blend -> alpha: 0.4 gamma: 0.7 mean: 0.049072041751249976


In [19]:
# BM3D v2: precise float normalization and lower sigma sweep
import bm3d

def normalize_divide_float_precise(img_u8: np.ndarray, ksize_bg: int = 31) -> np.ndarray:
    # All in float32: img_f / max(bg_f, eps)
    img_f = img_u8.astype(np.float32) / 255.0
    bg_u8 = estimate_background_median(img_u8, ksize=ksize_bg)
    bg_f = bg_u8.astype(np.float32) / 255.0
    eps = 1.0 / 255.0  # avoid zeros
    norm_f = img_f / np.maximum(bg_f, eps)
    norm_f = np.clip(norm_f, 0.0, 1.0).astype(np.float32)
    return norm_f

def estimate_sigma_mad_float_hpf(norm_f: np.ndarray, ksize: int = 5) -> float:
    # High-pass via Gaussian blur on float to avoid edge amplification
    k = ksize if ksize % 2 == 1 else ksize + 1
    blur = cv2.GaussianBlur(norm_f, (k, k), 0)
    hp = norm_f - blur
    med = float(np.median(hp))
    mad = float(np.median(np.abs(hp - med)))
    sigma_hat = 1.4826 * mad
    return float(np.clip(sigma_hat, 0.005, 0.05))

def pipeline_div_bm3d_v2(img_u8: np.ndarray, ksize_bg: int = 31, sigma_psd: float = 0.03) -> np.ndarray:
    norm_f = normalize_divide_float_precise(img_u8, ksize_bg=ksize_bg)
    den_f = bm3d.bm3d(norm_f, sigma_psd=sigma_psd, stage_arg=bm3d.BM3DStages.ALL_STAGES)
    den_f = np.clip(den_f, 0.0, 1.0).astype(np.float32)
    return den_f

def eval_bm3d_v2_global_sigma_on_train(sigmas: list[float], ksize_bg: int = 31):
    results = []
    for s in sigmas:
        errs = []
        t0 = time.time()
        print(f'[BM3D v2 global] sigma={s:.4f}')
        for i, p in enumerate(train_files):
            if i % 10 == 0:
                print(f'  sigma={s:.4f} img {i+1}/{len(train_files)} elapsed {time.time()-t0:.1f}s', flush=True)
            img = read_gray_uint8(p)
            den_f = pipeline_div_bm3d_v2(img, ksize_bg=ksize_bg, sigma_psd=s)
            target_f = to_float01(read_gray_uint8(TRAIN_CLEAN_DIR / p.name))
            errs.append(rmse(den_f, target_f))
        mean_rmse = float(np.mean(errs))
        med_rmse = float(np.median(errs))
        print(f'  -> mean RMSE={mean_rmse:.6f}, median={med_rmse:.6f}, time={time.time()-t0:.1f}s')
        results.append((s, mean_rmse, med_rmse))
    results.sort(key=lambda x: x[1])
    print('Top BM3D v2 global sigmas by mean RMSE:')
    for s, m, md in results[:5]:
        print(f'  sigma={s:.4f}: mean={m:.6f}, median={md:.6f}')
    return results

def eval_bm3d_v2_per_image_sigma_on_train(factors=(0.6, 0.8, 1.0), ksize_bg: int = 31):
    sums = {f: 0.0 for f in factors}
    n = 0
    t0_all = time.time()
    for i, p in enumerate(train_files):
        if i % 10 == 0:
            print(f'[BM3D v2 per-image] img {i+1}/{len(train_files)} elapsed {time.time()-t0_all:.1f}s', flush=True)
        img = read_gray_uint8(p)
        norm_f = normalize_divide_float_precise(img, ksize_bg=ksize_bg)
        sigma_hat = estimate_sigma_mad_float_hpf(norm_f, ksize=5)
        target_f = to_float01(read_gray_uint8(TRAIN_CLEAN_DIR / p.name))
        for f in factors:
            sigma = float(np.clip(sigma_hat * f, 0.005, 0.05))
            den_f = bm3d.bm3d(norm_f, sigma_psd=sigma, stage_arg=bm3d.BM3DStages.ALL_STAGES)
            den_f = np.clip(den_f, 0.0, 1.0).astype(np.float32)
            sums[f] += rmse(den_f, target_f)
        n += 1
    results = []
    for f in factors:
        results.append((f, float(sums[f]/n)))
    results.sort(key=lambda x: x[1])
    print('BM3D v2 per-image sigma multipliers by mean RMSE:')
    for f, m in results:
        print(f'  factor={f:.2f}: mean={m:.6f}')
    return results

# Quick eval with smaller sigmas first; if promising, we can expand
sigmas_v2 = [0.020, 0.025, 0.030, 0.035, 0.040]
bm3d_v2_global = eval_bm3d_v2_global_sigma_on_train(sigmas_v2, ksize_bg=31)
best_v2_sigma = bm3d_v2_global[0][0]
print('Best BM3D v2 global sigma:', best_v2_sigma)
# Optional per-image factor sweep (uncomment if global is close):
# bm3d_v2_perimg = eval_bm3d_v2_per_image_sigma_on_train(factors=(0.6, 0.8, 1.0), ksize_bg=31)
# best_v2_factor = bm3d_v2_perimg[0][0]
# print('Best BM3D v2 per-image factor:', best_v2_factor)

def generate_submission_bm3d_v2(out_path='submission.csv', decimals: int = 5, ksize_bg: int = 31, mode: str = 'global', global_sigma: float = 0.03, factor: float = 0.8):
    assert mode in ('global', 'per_image')
    fmt = '{:.' + str(decimals) + 'f}'
    sample_iter = pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000)
    ordered_image_ids = []
    seen = set()
    for chunk in sample_iter:
        for s in chunk['id'].astype(str).values:
            img_id, r, c = parse_id_triplet(s)
            if img_id not in seen:
                seen.add(img_id)
                ordered_image_ids.append(img_id)
    print(f'[BM3D v2 submit] Found {len(ordered_image_ids)} unique test image ids.')
    cache = {}
    for idx, img_id in enumerate(ordered_image_ids):
        img_path = TEST_DIR / f'{img_id}.png'
        print(f'[{idx+1}/{len(ordered_image_ids)}] {img_path}', flush=True)
        img_u8 = read_gray_uint8(img_path)
        norm_f = normalize_divide_float_precise(img_u8, ksize_bg=ksize_bg)
        if mode == 'global':
            sigma = float(global_sigma)
        else:
            sigma_hat = estimate_sigma_mad_float_hpf(norm_f, ksize=5)
            sigma = float(np.clip(sigma_hat * factor, 0.005, 0.05))
        den_f = bm3d.bm3d(norm_f, sigma_psd=sigma, stage_arg=bm3d.BM3DStages.ALL_STAGES)
        cache[img_id] = np.clip(den_f, 0.0, 1.0).astype(np.float32)
    print('Writing BM3D v2 predictions to CSV in sample order (fixed decimals)...')
    import csv
    with open(out_path, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['id', 'value'])
        for chunk in pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000):
            rows = []
            for s in chunk['id'].astype(str).values:
                img_id, r, c = parse_id_triplet(s)
                val = float(cache[img_id][r-1, c-1])
                rows.append((s, fmt.format(val)))
            writer.writerows(rows)
    print(f'Wrote: {out_path}')

[BM3D v2 global] sigma=0.0200
  sigma=0.0200 img 1/115 elapsed 0.0s


  sigma=0.0200 img 11/115 elapsed 17.1s


  sigma=0.0200 img 21/115 elapsed 34.1s


  sigma=0.0200 img 31/115 elapsed 51.1s


  sigma=0.0200 img 41/115 elapsed 68.7s


...

  sigma=0.0400 img 81/115 elapsed 156.5s


  sigma=0.0400 img 91/115 elapsed 178.5s


  sigma=0.0400 img 101/115 elapsed 200.4s


  sigma=0.0400 img 111/115 elapsed 222.4s


  -> mean RMSE=0.052493, median=0.050739, time=233.4s
Top BM3D v2 global sigmas by mean RMSE:
  sigma=0.0200: mean=0.051160, median=0.049039
  sigma=0.0250: mean=0.051318, median=0.049153
  sigma=0.0300: mean=0.051591, median=0.049544
  sigma=0.0350: mean=0.051981, median=0.050060
  sigma=0.0400: mean=0.052493, median=0.050739
Best BM3D v2 global sigma: 0.02


In [20]:
# Per-image adaptive background ksize selection {27,31,35} with RMSE-optimized thresholds
import numpy as np

def bg_ksize_candidates():
    return (27, 31, 35)

def normalize_with_ksize(img_u8: np.ndarray, ksize_bg: int) -> np.ndarray:
    bg = estimate_background_median(img_u8, ksize=ksize_bg)
    norm = normalize_divide(img_u8, bg)
    return norm

def stain_metric(img_u8: np.ndarray, ksize_ref: int = 31) -> float:
    # Use std of normalized image with reference bg ksize as a simple stain level proxy
    norm = normalize_with_ksize(img_u8, ksize_ref)
    return float(np.std(norm))

def collect_per_image_bg_errs(train_paths, ksizes=(27,31,35), h: int = 14, tmpl: int = 5, srch: int = 31, ksize_ref_metric: int = 31):
    per_img = []  # dict: {'path': p, 'metric': m, 'errs': {ks: err}, 'best_ks': ks, 'best_err': err}
    for i, p in enumerate(train_paths):
        if i % 10 == 0:
            print(f'collect bg errs: {i+1}/{len(train_paths)}', flush=True)
        img = read_gray_uint8(p)
        m = stain_metric(img, ksize_ref=ksize_ref_metric)
        target_f = to_float01(read_gray_uint8(TRAIN_CLEAN_DIR / p.name))
        errs_map = {}
        best_ks, best_err = None, 1e9
        for ks in ksizes:
            norm = normalize_with_ksize(img, ks)
            den = cv2.fastNlMeansDenoising(norm, None, h=h, templateWindowSize=tmpl, searchWindowSize=srch)
            err = rmse(to_float01(den), target_f)
            errs_map[ks] = err
            if err < best_err:
                best_err = err; best_ks = ks
        per_img.append({'path': p, 'metric': m, 'errs': errs_map, 'best_ks': best_ks, 'best_err': best_err})
    mean_oracle = float(np.mean([d['best_err'] for d in per_img]))
    print('Mean RMSE of oracle per-image bg-ksize:', mean_oracle)
    return per_img

def fit_metric_thresholds_min_rmse(per_img, ksizes=(27,31,35)):
    metrics = np.array([d['metric'] for d in per_img], dtype=np.float32)
    pcts = np.percentile(metrics, [20, 30, 40, 50, 60, 70, 80])
    best = None  # (mean_rmse, t1, t2)
    for i in range(len(pcts)):
        for j in range(i+1, len(pcts)):
            t1, t2 = float(pcts[i]), float(pcts[j])
            errs = []
            for d in per_img:
                s = d['metric']
                if s < t1: ks = ksizes[0]
                elif s < t2: ks = ksizes[1]
                else: ks = ksizes[2]
                errs.append(d['errs'][ks])
            mean_rmse = float(np.mean(errs))
            if best is None or mean_rmse < best[0]:
                best = (mean_rmse, t1, t2)
    print(f'Best bg-ksize thresholds by RMSE: mean={best[0]:.6f} at t1={best[1]:.6f}, t2={best[2]:.6f}')
    return best[1], best[2], best[0]

def eval_bg_threshold_assignment_from_cache(per_img, t1: float, t2: float, ksizes=(27,31,35)):
    errs = []
    for d in per_img:
        s = d['metric']
        if s < t1: ks = ksizes[0]
        elif s < t2: ks = ksizes[1]
        else: ks = ksizes[2]
        errs.append(d['errs'][ks])
    mean_rmse = float(np.mean(errs)); med_rmse = float(np.median(errs))
    print(f'Cached bg-ksize assignment RMSE -> mean={mean_rmse:.6f}, median={med_rmse:.6f}')
    return mean_rmse, med_rmse

def generate_submission_per_image_bgksize(t1: float, t2: float, ksizes=(27,31,35), h: int = 14, tmpl: int = 5, srch: int = 31, out_path='submission.csv', decimals: int = 5):
    fmt = '{:.' + str(decimals) + 'f}'
    # Ordered unique image ids
    sample_iter = pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000)
    ordered_image_ids, seen = [], set()
    for chunk in sample_iter:
        for s in chunk['id'].astype(str).values:
            img_id, r, c = parse_id_triplet(s)
            if img_id not in seen:
                seen.add(img_id); ordered_image_ids.append(img_id)
    print(f'Found {len(ordered_image_ids)} unique test image ids for bg-ksize selection.')
    cache = {}
    for idx, img_id in enumerate(ordered_image_ids):
        img_path = TEST_DIR / f'{img_id}.png'
        print(f'[{idx+1}/{len(ordered_image_ids)}] {img_path}', flush=True)
        img_u8 = read_gray_uint8(img_path)
        m = stain_metric(img_u8, ksize_ref=31)
        if m < t1: ks = ksizes[0]
        elif m < t2: ks = ksizes[1]
        else: ks = ksizes[2]
        norm = normalize_with_ksize(img_u8, ks)
        den = cv2.fastNlMeansDenoising(norm, None, h=h, templateWindowSize=tmpl, searchWindowSize=srch)
        cache[img_id] = to_float01(den)
    # Write with fixed decimals
    import csv
    print(f'Writing per-image bg-ksize predictions to CSV ({decimals} decimals)...')
    with open(out_path, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['id', 'value'])
        for chunk in pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000):
            rows = []
            for s in chunk['id'].astype(str).values:
                img_id, r, c = parse_id_triplet(s)
                val = float(np.clip(cache[img_id][r-1, c-1], 0.0, 1.0))
                rows.append((s, fmt.format(val)))
            writer.writerows(rows)
    print(f'Wrote: {out_path}')

# Run per-image bg-ksize oracle and threshold fit
ks_cands = bg_ksize_candidates()
per_img_bg = collect_per_image_bg_errs(train_files, ksizes=ks_cands, h=14, tmpl=5, srch=31, ksize_ref_metric=31)
t1_bg, t2_bg, best_mean_bg = fit_metric_thresholds_min_rmse(per_img_bg, ksizes=ks_cands)
mean_rmse_bg, med_rmse_bg = eval_bg_threshold_assignment_from_cache(per_img_bg, t1_bg, t2_bg, ksizes=ks_cands)
print('bg-ksize thresholds:', t1_bg, t2_bg, '-> mean RMSE:', mean_rmse_bg, 'median RMSE:', med_rmse_bg)
# If mean_rmse_bg < 0.049001, generate a submission:
# generate_submission_per_image_bgksize(t1_bg, t2_bg, ksizes=ks_cands, h=14, tmpl=5, srch=31, out_path='submission.csv', decimals=5)

collect bg errs: 1/115


collect bg errs: 11/115


collect bg errs: 21/115


collect bg errs: 31/115


collect bg errs: 41/115


...

collect bg errs: 81/115


collect bg errs: 91/115


collect bg errs: 101/115


collect bg errs: 111/115


Mean RMSE of oracle per-image bg-ksize: 0.04843027288823024
Best bg-ksize thresholds by RMSE: mean=0.048763 at t1=62.757529, t2=63.837418
Cached bg-ksize assignment RMSE -> mean=0.048763, median=0.047317
bg-ksize thresholds: 62.757529449462886 63.83741760253906 -> mean RMSE: 0.04876299471310947 median RMSE: 0.04731684550642967


In [21]:
# Generate submission using adaptive background ksize thresholds found above (5 decimals)
print('Using bg-ksize thresholds:', t1_bg, t2_bg)
generate_submission_per_image_bgksize(t1_bg, t2_bg, ksizes=ks_cands, h=14, tmpl=5, srch=31, out_path='submission.csv', decimals=5)

Using bg-ksize thresholds: 62.757529449462886 63.83741760253906


Found 29 unique test image ids for bg-ksize selection.
[1/29] test/110.png


[2/29] test/111.png


[3/29] test/122.png


[4/29] test/131.png


...

[27/29] test/8.png


[28/29] test/80.png


[29/29] test/95.png


Writing per-image bg-ksize predictions to CSV (5 decimals)...


Wrote: submission.csv


In [22]:
# Float-based normalization with background floor + post percentile stretch (DDD tricks) using NLM (uint8) in the middle
import numpy as np

def normalize_divide_float_with_floor(img_u8: np.ndarray, ksize_bg: int = 31, bg_floor_u8: int = 5) -> tuple[np.ndarray, np.ndarray]:
    # Compute float normalization with background floor, then quantize to uint8 for OpenCV NLM
    bg = estimate_background_median(img_u8, ksize=ksize_bg)
    img_f = img_u8.astype(np.float32) / 255.0
    bg_f = bg.astype(np.float32) / 255.0
    floor = float(bg_floor_u8) / 255.0
    bg_safe = np.maximum(bg_f, floor)
    norm_f = (img_f / bg_safe).clip(0.0, 1.0).astype(np.float32)
    norm_u8 = (norm_f * 255.0).clip(0, 255).astype(np.uint8)
    return norm_u8, norm_f

def percentile_stretch_float(img_f: np.ndarray, p_low: float = 1.0, p_high: float = 99.0) -> np.ndarray:
    lo = float(np.percentile(img_f, p_low))
    hi = float(np.percentile(img_f, p_high))
    if hi <= lo + 1e-6:
        return np.clip(img_f, 0.0, 1.0).astype(np.float32)
    out = (img_f - lo) / (hi - lo)
    return np.clip(out, 0.0, 1.0).astype(np.float32)

def pipeline_floatnorm_nlm_pstretch(
    img_u8: np.ndarray,
    ksize_bg: int = 31,
    bg_floor_u8: int = 5,
    h: int = 14,
    tmpl: int = 5,
    srch: int = 31,
    p_low: float = 1.0,
    p_high: float = 99.0
) -> np.ndarray:
    norm_u8, _ = normalize_divide_float_with_floor(img_u8, ksize_bg=ksize_bg, bg_floor_u8=bg_floor_u8)
    den_u8 = cv2.fastNlMeansDenoising(norm_u8, None, h=h, templateWindowSize=tmpl, searchWindowSize=srch)
    den_f = (den_u8.astype(np.float32) / 255.0).clip(0.0, 1.0)
    out_f = percentile_stretch_float(den_f, p_low=p_low, p_high=p_high)
    return out_f

def eval_floatnorm_pstretch_on_train(
    bg_floor_list=(3, 5, 7), p_low_list=(0.5, 1.0, 1.5), p_high: float = 99.0,
    ksize_bg: int = 31, h: int = 14, tmpl: int = 5, srch: int = 31
):
    results = []
    for floor_u8 in bg_floor_list:
        for p_low in p_low_list:
            errs = []
            t0 = time.time()
            print(f'[float+stretch] floor={floor_u8}, p_low={p_low}, p_high={p_high}')
            for i, p in enumerate(train_files):
                if i % 10 == 0:
                    print(f'  img {i+1}/{len(train_files)} elapsed {time.time()-t0:.1f}s', flush=True)
                img = read_gray_uint8(p)
                pred_f = pipeline_floatnorm_nlm_pstretch(img, ksize_bg=ksize_bg, bg_floor_u8=floor_u8, h=h, tmpl=tmpl, srch=srch, p_low=p_low, p_high=p_high)
                target_f = to_float01(read_gray_uint8(TRAIN_CLEAN_DIR / p.name))
                errs.append(rmse(pred_f, target_f))
            mean_rmse = float(np.mean(errs))
            med_rmse = float(np.median(errs))
            print(f'  -> mean RMSE={mean_rmse:.6f}, median RMSE={med_rmse:.6f}, time={time.time()-t0:.1f}s')
            results.append((floor_u8, p_low, p_high, mean_rmse, med_rmse))
    results.sort(key=lambda x: x[3])
    print('Top float+stretch configs by mean RMSE:')
    for floor_u8, p_low, p_high, m, md in results[:5]:
        print(f'  floor={floor_u8}, p_low={p_low}, p_high={p_high}: mean={m:.6f}, median={md:.6f}')
    return results

def generate_submission_floatnorm_pstretch(
    bg_floor_u8: int, p_low: float, p_high: float,
    ksize_bg: int = 31, h: int = 14, tmpl: int = 5, srch: int = 31,
    out_path: str = 'submission.csv', decimals: int = 5
):
    fmt = '{:.' + str(decimals) + 'f}'
    # Ordered unique image ids
    sample_iter = pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000)
    ordered_image_ids, seen = [], set()
    for chunk in sample_iter:
        for s in chunk['id'].astype(str).values:
            img_id, r, c = parse_id_triplet(s)
            if img_id not in seen:
                seen.add(img_id); ordered_image_ids.append(img_id)
    print(f'[submit float+stretch] Found {len(ordered_image_ids)} unique test image ids.')
    cache = {}
    for idx, img_id in enumerate(ordered_image_ids):
        img_path = TEST_DIR / f'{img_id}.png'
        print(f'[{idx+1}/{len(ordered_image_ids)}] {img_path}', flush=True)
        img_u8 = read_gray_uint8(img_path)
        pred_f = pipeline_floatnorm_nlm_pstretch(img_u8, ksize_bg=ksize_bg, bg_floor_u8=bg_floor_u8, h=h, tmpl=tmpl, srch=srch, p_low=p_low, p_high=p_high)
        cache[img_id] = pred_f.astype(np.float32)
    # Write in sample order with fixed decimals
    import csv
    print(f'Writing predictions to CSV ({decimals} decimals)...')
    with open(out_path, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['id', 'value'])
        for chunk in pd.read_csv(SAMPLE_SUB_PATH, chunksize=200000):
            rows = []
            for s in chunk['id'].astype(str).values:
                img_id, r, c = parse_id_triplet(s)
                val = float(np.clip(cache[img_id][r-1, c-1], 0.0, 1.0))
                rows.append((s, fmt.format(val)))
            writer.writerows(rows)
    print(f'Wrote: {out_path}')

# Evaluate float normalization + percentile stretch on train
float_stretch_results = eval_floatnorm_pstretch_on_train(bg_floor_list=(3,5,7), p_low_list=(0.5,1.0,1.5), p_high=99.0, ksize_bg=31, h=14, tmpl=5, srch=31)
best_floor, best_plow, best_phigh, best_m, best_md = float_stretch_results[0]
print('Best float+stretch config -> floor:', best_floor, 'p_low:', best_plow, 'p_high:', best_phigh, 'mean:', best_m)
# If best_m < current best (0.048763), consider generating a submission:
# generate_submission_floatnorm_pstretch(bg_floor_u8=int(best_floor), p_low=float(best_plow), p_high=float(best_phigh), ksize_bg=31, h=14, tmpl=5, srch=31, out_path='submission.csv', decimals=5)

[float+stretch] floor=3, p_low=0.5, p_high=99.0
  img 1/115 elapsed 0.0s


  img 11/115 elapsed 3.8s


  img 21/115 elapsed 7.6s


  img 31/115 elapsed 11.4s


  img 41/115 elapsed 15.1s


...

  img 81/115 elapsed 27.4s


  img 91/115 elapsed 30.6s


  img 101/115 elapsed 33.8s


  img 111/115 elapsed 36.9s


  -> mean RMSE=0.039089, median RMSE=0.036856, time=38.5s
Top float+stretch configs by mean RMSE:
  floor=3, p_low=1.5, p_high=99.0: mean=0.039089, median=0.036856
  floor=5, p_low=1.5, p_high=99.0: mean=0.039089, median=0.036856
  floor=7, p_low=1.5, p_high=99.0: mean=0.039089, median=0.036856
  floor=3, p_low=1.0, p_high=99.0: mean=0.039264, median=0.037116
  floor=5, p_low=1.0, p_high=99.0: mean=0.039264, median=0.037116
Best float+stretch config -> floor: 3 p_low: 1.5 p_high: 99.0 mean: 0.03908857217301493


In [23]:
# Generate submission using best float-based normalization + percentile stretch config
print('Generating submission with float normalization + percentile stretch...')
best_floor, best_plow, best_phigh = 3, 1.5, 99.0
generate_submission_floatnorm_pstretch(bg_floor_u8=int(best_floor), p_low=float(best_plow), p_high=float(best_phigh), ksize_bg=31, h=14, tmpl=5, srch=31, out_path='submission.csv', decimals=5)

Generating submission with float normalization + percentile stretch...


[submit float+stretch] Found 29 unique test image ids.
[1/29] test/110.png


[2/29] test/111.png


[3/29] test/122.png


[4/29] test/131.png


...

[27/29] test/8.png


[28/29] test/80.png


[29/29] test/95.png


Writing predictions to CSV (5 decimals)...


Wrote: submission.csv


In [24]:
# Regenerate submission with fewer decimals (4) to fit size limit < 100MB
print('Regenerating submission with 4 decimals to reduce file size...')
best_floor, best_plow, best_phigh = 3, 1.5, 99.0
generate_submission_floatnorm_pstretch(bg_floor_u8=int(best_floor), p_low=float(best_plow), p_high=float(best_phigh), ksize_bg=31, h=14, tmpl=5, srch=31, out_path='submission.csv', decimals=4)

Regenerating submission with 4 decimals to reduce file size...


[submit float+stretch] Found 29 unique test image ids.
[1/29] test/110.png


[2/29] test/111.png


[3/29] test/122.png


[4/29] test/131.png


...

[27/29] test/8.png


[28/29] test/80.png


[29/29] test/95.png


Writing predictions to CSV (4 decimals)...


Wrote: submission.csv


In [25]:
# Regenerate submission with 3 decimals to ensure file size < 100MB
print('Regenerating submission with 3 decimals to reduce file size below limit...')
best_floor, best_plow, best_phigh = 3, 1.5, 99.0
generate_submission_floatnorm_pstretch(bg_floor_u8=int(best_floor), p_low=float(best_plow), p_high=float(best_phigh), ksize_bg=31, h=14, tmpl=5, srch=31, out_path='submission.csv', decimals=3)

Regenerating submission with 3 decimals to reduce file size below limit...


[submit float+stretch] Found 29 unique test image ids.
[1/29] test/110.png


[2/29] test/111.png


[3/29] test/122.png


[4/29] test/131.png


...

[27/29] test/8.png


[28/29] test/80.png


[29/29] test/95.png


Writing predictions to CSV (3 decimals)...


Wrote: submission.csv
